In [ ]:
import ipywidgets as widgets
from IPython.display import display
from pybela import Streamer
import biquad
from callback import callback, CallbackState

In [ ]:
vars = ['gFaabSensor_1', 'gFaabSensor_2', 'gFaabSensor_3', 'gFaabSensor_4', 'gFaabSensor_5', 'gFaabSensor_6', 'gFaabSensor_7', 'gFaabSensor_8']

streamer = Streamer()
streamer.connect()

In [ ]:
cs = CallbackState(
    seq_len=512,
    num_models=20,
    num_blocks_to_compute_avg=10,
    num_blocks_to_compute_std=40,
    filter=biquad.lowpass(sr=streamer.sample_rate, f=1, q=0.707),
    num_of_iterations_in_this_model_check=20,
    init_ratio_rising_threshold=2.5,
    init_ratio_falling_threshold=1.3,
    threshold_leak=0.1,
    trigger_width=25,
    trigger_idx=4,
    running_norm=True,
    permute_out=False,
    path="models/trained"
)

In [ ]:
streamer.start_streaming(vars, on_block_callback=callback, callback_args=(cs,streamer))

In [ ]:
streamer.stop_streaming()

In [ ]:
# Create sliders using values from 'gain' list
gain_sliders = [widgets.FloatSlider(value=gain[i], min=0.0, max=2.0, step=0.01, description=f'Gain {i+1}') for i in range(len(gain))]

# Update function to adjust gain parameters
def update_gain(change):
    global gain
    gain = [slider.value for slider in gain_sliders]
    # Optional: Call a function here to apply the new gain values immediately

# Display widgets and link them to the update function
slider_box = widgets.VBox([widgets.Label('Adjust Gain Parameters:')] + gain_sliders)

In [ ]:
display(slider_box)

for slider in gain_sliders:
    slider.observe(update_gain, names='value')

# plot

In [ ]:
import os
import bokeh
import bokeh.plotting
import bokeh.io
import bokeh.driving
from bokeh.resources import INLINE

os.environ["BOKEH_ALLOW_WS_ORIGIN"]="0j0t0jnmqu776ei6png0k89bho2qg0m6ia345511p3uleiqq2kep"

In [ ]:
streamer.start_streaming(variables=["gFaabSensor_1", "gFaabSensor_2"])

In [ ]:
streamer.plot_data(x_var="gFaabSensor_1", y_vars=["gFaabSensor_1", "gFaabSensor_2"], y_range=[0, 1], rollover=10000)

In [ ]:
plot_data = {
    
    "out_1": {"timestamps": [0,1,2], "data" : [0.1, 0.2, 0.3]},
    "out_2": {"timestamps": [0,1,2], "data" : [0.1, 0.2, 0.3]},
}

In [ ]:
from itertools import cycle
import asyncio

def _bokeh_plot_data_app(
                            data,
                            x_var,
                            y_vars,
                            y_range=None,
                            rollover=None,
                            plot_update_delay=90):

    def _app(doc):
        # Instantiate figures
        p = bokeh.plotting.figure(
            frame_width=500,
            frame_height=175,
            x_axis_label="timestamps",
            y_axis_label="value",
        )

        if y_range is not None:
            p.y_range = bokeh.models.Range1d(y_range[0], y_range[1])

        # No padding on x_range makes data flush with end of plot
        p.x_range.range_padding = 0

        # Create a dictionary to store ColumnDataSource instances for each y_var
        template = {"timestamps": [], **{var: [] for var in data}}
        source = bokeh.models.ColumnDataSource(template)

        # # Create line glyphs for each y_var
        colors = cycle([
            "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728",
            "#9467bd", "#8c564b", "#e377c2", "#7f7f7f",
            "#bcbd22", "#17becf", "#1a55FF", "#FF1A1A"
        ])
        for y_var in y_vars:
            p.line(source=source, x="timestamps",
                    y=y_var, line_color=next(colors), legend_label=y_var)

        @bokeh.driving.linear()
        def update(step):
            # Update plot by streaming in data
            new_data = {"timestamps": [
                data[x_var]["timestamp"]]if "timestamp" in data[x_var] else data[x_var]["timestamps"]}
            for y_var in y_vars:
                new_data[y_var] = data[y_var]["data"] if isinstance(
                    data[y_var]["data"], list) else [data[y_var]["data"]]
            source.stream(new_data, rollover)

        doc.add_root(p)
        doc.add_periodic_callback(update, plot_update_delay)
    return _app

def plot_data( x_var, y_vars, y_range=None, plot_update_delay=100, rollover=500):

    # wait until streaming buffers have been populated
    async def wait_for_streaming_buffers_to_arrive():
        while not all(data['data'] for data in {
                var: _buffer for var, _buffer in last_streamed_buffer.items() if var in y_vars}.values()):
            await asyncio.sleep(0.1)
    asyncio.run(wait_for_streaming_buffers_to_arrive())

    bokeh.io.output_notebook(INLINE)
    bokeh.io.show(_bokeh_plot_data_app(data={
        var: _buffer for var, _buffer in last_streamed_buffer.items() if var in y_vars}, x_var=x_var,
        y_vars=y_vars, y_range=y_range, plot_update_delay=plot_update_delay, rollover=rollover))


In [ ]:
plot_data(x_var="gFaabSensor_1", y_vars=["gFaabSensor_1", "gFaabSensor_2"], y_range=[0, 1], rollover=10000)

In [ ]:
streamer.stop_streaming()